In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import requests
import json

import time
import re
import copy

from crpapi import CRP
from congress import Congress

## Support Functions

In [3]:
def get_keys(path):
    """
    Pulls necessary api keys from designated path
    """
    with open(path) as f:
        return json.load(f)

## ETL

In [4]:
current_congress = 116
start_congress = 113

### Voteview

In [5]:
def get_voteview_csv_content(content_type, current_congress_num):
    '''
    Retrieves congress data by content type (passed in) for the 104th Congress to the current Congress today.
    
    '''
    if content_type not in ['votes', 'rollcalls', 'members']:
        print('Not a valid content type')
        return
    
    for congress_num in range(104, current_congress_num+1):
        #download the vote each member made for each roll call
        url = 'https://voteview.com/static/data/out/{}/S{}_{}.csv'.format(content_type, congress_num, content_type)

        r = requests.get(url)
        content = r.content
        file = open('{}/S{}_{}.csv'.format(content_type, congress_num, content_type), 'wb')

        file.write(content)
        file.close()
        
        #put in a sleep timer to not overload the voteview servers
        time.sleep(1)
        
    return

In [6]:
# get_voteview_csv_content('votes', current_congress)

In [7]:
# get_voteview_csv_content('rollcalls', current_congress)

In [8]:
# get_voteview_csv_content('members', current_congress)

In [9]:
rollcall_df = pd.DataFrame()
for i in range(start_congress, current_congress+1):
    temp_df = pd.read_csv('rollcalls/S{}_rollcalls.csv'.format(i))
    rollcall_df = pd.concat([rollcall_df, temp_df], ignore_index=True)
    
rollcall_df.drop('dtl_desc', axis=1, inplace=True)
rollcall_df.dropna(axis=0, inplace=True)
    
display(rollcall_df.head())
display(rollcall_df.tail())
display(rollcall_df.info())

,congress,chamber,rollnumber,date,session,clerk_rollnumber,yea_count,nay_count,nominate_mid_1,nominate_mid_2,nominate_spread_1,nominate_spread_2,nominate_log_likelihood,bill_number,vote_result,vote_desc,vote_question
0,113,Senate,1,2013-01-24,1,1,78,16,0.522,-0.852,0.214,-0.031,-20.316,SRES15,Resolution Agreed to,A resolution to improve procedures for the con...,On the Resolution
1,113,Senate,2,2013-01-24,1,2,86,9,0.620,-0.785,0.201,-0.111,-17.612,SRES16,Resolution Agreed to,A resolution amending the Standing Rules of th...,On the Resolution
2,113,Senate,3,2013-01-28,1,3,35,62,0.277,-0.961,-0.305,0.050,-18.941,HR152,Amendment Rejected,To offset the cost of the bill with rescission...,On the Amendment
3,113,Senate,4,2013-01-28,1,4,63,36,0.195,-0.812,0.460,-0.392,-15.561,HR152,Bill Passed,A bill making supplemental appropriations for ...,On Passage of the Bill
4,113,Senate,5,2013-01-29,1,5,95,3,0.998,-0.068,1.749,-0.128,-9.990,PN42,Nomination Confirmed,"John Forbes Kerry, of Massachusetts, to be Sec...",On the Nomination


,congress,chamber,rollnumber,date,session,clerk_rollnumber,yea_count,nay_count,nominate_mid_1,nominate_mid_2,nominate_spread_1,nominate_spread_2,nominate_log_likelihood,bill_number,vote_result,vote_desc,vote_question
2261,116,Senate,504,2020-03-18,2,76,90,8,0.544,-0.839,0.268,-0.311,-13.629,HR6201,Bill Passed,A bill making emergency supplemental appropria...,On Passage of the Bill
2262,116,Senate,505,2020-03-22,2,77,47,47,-0.009,-0.717,-0.352,0.347,-5.644,HR748,Cloture on the Motion to Proceed Rejected,A bill to amend the Internal Revenue Code of 1...,On Cloture on the Motion to Proceed
2263,116,Senate,506,2020-03-23,2,78,49,46,-0.037,-0.497,-0.720,0.022,-2.314,HR748,Cloture on the Motion to Proceed Rejected,A bill to amend the Internal Revenue Code of 1...,On Cloture on the Motion to Proceed
2264,116,Senate,507,2020-03-25,2,79,48,48,-0.243,0.819,-0.516,-1.138,-6.480,HR748,Amendment Rejected,To ensure that additional unemployment benefit...,On the Amendment
2265,116,Senate,508,2020-03-25,2,80,96,0,0.000,0.000,0.000,0.000,0.000,HR748,Bill Passed,A bill to amend the Internal Revenue Code of 1...,On Passage of the Bill


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2263 entries, 0 to 2265
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   congress                 2263 non-null   int64  
 1   chamber                  2263 non-null   object 
 2   rollnumber               2263 non-null   int64  
 3   date                     2263 non-null   object 
 4   session                  2263 non-null   int64  
 5   clerk_rollnumber         2263 non-null   int64  
 6   yea_count                2263 non-null   int64  
 7   nay_count                2263 non-null   int64  
 8   nominate_mid_1           2263 non-null   float64
 9   nominate_mid_2           2263 non-null   float64
 10  nominate_spread_1        2263 non-null   float64
 11  nominate_spread_2        2263 non-null   float64
 12  nominate_log_likelihood  2263 non-null   float64
 13  bill_number              2263 non-null   object 
 14  vote_result             

None

In [86]:
rollcall_df[rollcall_df['bill_number'].str.contains('TREATYDOC')]

,congress,chamber,rollnumber,date,session,clerk_rollnumber,yea_count,nay_count,nominate_mid_1,nominate_mid_2,nominate_spread_1,nominate_spread_2,nominate_log_likelihood,bill_number,vote_result,vote_desc,vote_question
1255,115,Senate,97,2017-03-27,1,97,97,2,0.000,0.000,0.000,0.000,0.000,TREATYDOC11412,Cloture Motion Agreed to,Protocol to the North Atlantic Treaty of 1949 ...,On the Cloture Motion
1256,115,Senate,98,2017-03-28,1,98,97,2,0.000,0.000,0.000,0.000,0.000,TREATYDOC11412,Resolution of Ratification Agreed to,Protocol to the North Atlantic Treaty of 1949 ...,On the Resolution of Ratification
1963,116,Senate,206,2019-07-16,1,206,94,1,0.000,0.000,0.000,0.000,0.000,TREATYDOC1134,Cloture Motion Agreed to,The Protocol Amending the Convention between t...,On the Cloture Motion
1964,116,Senate,207,2019-07-16,1,207,4,92,-1.000,0.011,0.353,0.468,-9.787,TREATYDOC1134,Amendment Rejected,To amend the Protocol to protect tax privacy.,On the Amendment
1965,116,Senate,208,2019-07-16,1,208,4,92,-1.000,0.011,0.353,0.468,-9.787,TREATYDOC1134,Amendment Rejected,To provide a reservation to the Protocol.,On the Amendment
1966,116,Senate,209,2019-07-16,1,209,94,2,0.000,0.000,0.000,0.000,0.000,TREATYDOC1134,Resolution of Ratification Agreed to,The Protocol Amending the Convention between t...,On the Resolution of Ratification
1967,116,Senate,210,2019-07-17,1,210,95,2,0.000,0.000,0.000,0.000,0.000,TREATYDOC1121,Resolution of Ratification Agreed to,Protocol Amending the Convention between the U...,On the Resolution of Ratification
1968,116,Senate,211,2019-07-17,1,211,95,2,0.000,0.000,0.000,0.000,0.000,TREATYDOC1141,Resolution of Ratification Agreed to,The Protocol Amending the Convention between t...,On the Resolution of Ratification
1969,116,Senate,212,2019-07-17,1,212,93,3,-0.719,0.696,-0.255,0.068,-10.387,TREATYDOC1118,Resolution of Ratification Agreed to,Protocol Amending the Convention between the G...,On the Resolution of Ratification
2083,116,Senate,326,2019-10-21,1,326,84,2,0.000,0.000,0.000,0.000,0.000,TREATYDOC1161,Cloture Motion Agreed to,Protocol to the North Atlantic Treaty of 1949 ...,On the Cloture Motion


In [85]:
rollcall_df[~rollcall_df['bill_number'].str.contains('PN')]

,congress,chamber,rollnumber,date,session,clerk_rollnumber,yea_count,nay_count,nominate_mid_1,nominate_mid_2,nominate_spread_1,nominate_spread_2,nominate_log_likelihood,bill_number,vote_result,vote_desc,vote_question
0,113,Senate,1,2013-01-24,1,1,78,16,0.522,-0.852,0.214,-0.031,-20.316,SRES15,Resolution Agreed to,A resolution to improve procedures for the con...,On the Resolution
1,113,Senate,2,2013-01-24,1,2,86,9,0.620,-0.785,0.201,-0.111,-17.612,SRES16,Resolution Agreed to,A resolution amending the Standing Rules of th...,On the Resolution
2,113,Senate,3,2013-01-28,1,3,35,62,0.277,-0.961,-0.305,0.050,-18.941,HR152,Amendment Rejected,To offset the cost of the bill with rescission...,On the Amendment
3,113,Senate,4,2013-01-28,1,4,63,36,0.195,-0.812,0.460,-0.392,-15.561,HR152,Bill Passed,A bill making supplemental appropriations for ...,On Passage of the Bill
5,113,Senate,6,2013-01-31,1,6,54,44,-0.218,-0.976,0.358,-0.527,-8.469,HR325,Motion to Table Agreed to,To require that any debt limit increase be bal...,On the Motion to Table
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2261,116,Senate,504,2020-03-18,2,76,90,8,0.544,-0.839,0.268,-0.311,-13.629,HR6201,Bill Passed,A bill making emergency supplemental appropria...,On Passage of the Bill
2262,116,Senate,505,2020-03-22,2,77,47,47,-0.009,-0.717,-0.352,0.347,-5.644,HR748,Cloture on the Motion to Proceed Rejected,A bill to amend the Internal Revenue Code of 1...,On Cloture on the Motion to Proceed
2263,116,Senate,506,2020-03-23,2,78,49,46,-0.037,-0.497,-0.720,0.022,-2.314,HR748,Cloture on the Motion to Proceed Rejected,A bill to amend the Internal Revenue Code of 1...,On Cloture on the Motion to Proceed
2264,116,Senate,507,2020-03-25,2,79,48,48,-0.243,0.819,-0.516,-1.138,-6.480,HR748,Amendment Rejected,To ensure that additional unemployment benefit...,On the Amendment


In [80]:
unique_116_votes = rollcall_df[rollcall_df['congress'] == 116].bill_number.unique()

just_116_bills = [x for x in unique_116_votes if 'PN' not in x]
just_116_bills

['S1',
 'SJRES2',
 'S109',
 'HR268',
 'S47',
 'HJRES31',
 'S311',
 'SJRES7',
 'HJRES46',
 'SJRES8',
 'SRES50',
 'S151',
 'HR2157',
 'S1332',
 'SRES212',
 'SJRES20',
 'SJRES26',
 'S1790',
 'SJRES36',
 'SJRES38',
 'SJRES48',
 'HR3401',
 'TREATYDOC1134',
 'TREATYDOC1121',
 'TREATYDOC1141',
 'TREATYDOC1118',
 'HR1327',
 'SJRES37',
 'HR3877',
 'HR2740',
 'SJRES54',
 'SRES331',
 'SRES332',
 'SRES333',
 'SRES335',
 'SRES336',
 'HR4378',
 'SJRES53',
 'TREATYDOC1161',
 'HR3055',
 'SJRES50',
 'SJRES52',
 'S2740',
 'HR2333',
 'HR1865',
 'HR1158',
 'HR5430',
 'SRES483',
 'SRES488',
 'HRES755',
 'SJRES68',
 'S3275',
 'S2657',
 'HR6074',
 'SJRES56',
 'HJRES76',
 'HR6201',
 'HR748']

In [10]:
votes_to_ignore = rollcall_df[rollcall_df['bill_number'].str.contains('PN')].bill_number.unique()

non_nomination_votes = rollcall_df[~rollcall_df['bill_number'].isin(votes_to_ignore)][['congress', 'bill_number']]

non_nomination_votes['bill_id'] = non_nomination_votes['bill_number'].apply(lambda x: x.lower()) + '-' + \
                                  non_nomination_votes['congress'].astype(str)

bills_to_search = non_nomination_votes['bill_id'].unique()
bills_to_search

array(['sres15-113', 'sres16-113', 'hr152-113', 'hr325-113', 's47-113',
       's16-113', 's388-113', 'sres64-113', 'hr933-113', 'sconres8-113',
       's649-113', 's743-113', 's601-113', 's954-113', 'sres65-113',
       's1003-113', 's953-113', 's744-113', 's1238-113', 's1243-113',
       'hr1911-113', 'hr527-113', 'hjres59-113', 's1569-113',
       'hr2775-113', 'sjres26-113', 's815-113', 'hr3204-113', 's1197-113',
       'sconres28-113', 'hr3304-113', 's1845-113', 'hjres106-113',
       'hr3547-113', 's1926-113', 'hr2642-113', 's1963-113', 's540-113',
       's25-113', 's1982-113', 's1752-113', 's1917-113', 's1086-113',
       'hr3370-113', 's2124-113', 'hr4152-113', 'hr3979-113',
       'hr4302-113', 's2199-113', 's2223-113', 's2262-113', 'hr3474-113',
       'hr3080-113', 's2432-113', 'hr3230-113', 'hr4660-113', 'hr803-113',
       's2363-113', 's2578-113', 's2244-113', 's2569-113', 'hr5021-113',
       's2648-113', 'sjres19-113', 'hjres124-113', 's2280-113',
       's2685-113', '

In [95]:
member_df = pd.DataFrame()
for i in range(start_congress, current_congress+1):
    temp_df = pd.read_csv('members/S{}_members.csv'.format(i))
    member_df = pd.concat([member_df, temp_df], ignore_index=True)
    
president_indexes = member_df[member_df['chamber'] == 'President'].index.values
            
member_df.drop(president_indexes, inplace=True)
    
display(member_df.head())
display(member_df.tail())
display(member_df.info())

,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,bioname,...,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
1,113,Senate,49700,41,0,AL,200,0.0,1.0,"SESSIONS, Jefferson Beauregard III (Jeff)",...,NaN,0.549,0.130,-101.80298,0.82789,539.0,45.0,NaN,0.619,0.082
2,113,Senate,94659,41,0,AL,200,0.0,1.0,"SHELBY, Richard C.",...,NaN,0.428,0.514,-111.12805,0.81554,545.0,55.0,NaN,0.515,0.602
3,113,Senate,40300,81,0,AK,200,0.0,1.0,"MURKOWSKI, Lisa",...,NaN,0.210,-0.289,-158.34478,0.73575,516.0,83.0,NaN,0.124,-0.411
4,113,Senate,40900,81,0,AK,100,0.0,1.0,"BEGICH, Mark",...,NaN,-0.235,0.150,-48.13809,0.91000,512.0,21.0,NaN,-0.241,0.348
5,113,Senate,15039,61,0,AZ,200,0.0,1.0,"McCAIN, John Sidney, III",...,2018.0,0.381,-0.626,-132.74405,0.77659,525.0,67.0,NaN,0.412,-0.579


,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,bioname,...,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
408,116,Senate,40915,56,0,WV,100,NaN,NaN,"MANCHIN, Joe, III",...,NaN,-0.055,0.444,-44.94991,0.90671,459.0,10.0,NaN,-0.045,0.368
409,116,Senate,29940,25,0,WI,100,NaN,NaN,"BALDWIN, Tammy",...,NaN,-0.498,-0.179,-95.06820,0.81402,462.0,47.0,NaN,-0.396,-0.057
410,116,Senate,41111,25,0,WI,200,NaN,NaN,"JOHNSON, Ron",...,NaN,0.603,-0.288,-35.79249,0.92338,449.0,17.0,NaN,0.599,-0.057
411,116,Senate,40707,68,0,WY,200,NaN,NaN,"BARRASSO, John A.",...,NaN,0.539,0.237,-22.99352,0.95155,463.0,14.0,NaN,0.601,0.231
412,116,Senate,49706,68,0,WY,200,NaN,NaN,"ENZI, Michael B.",...,NaN,0.544,0.192,-34.94596,0.92684,460.0,13.0,NaN,0.580,0.336


<class 'pandas.core.frame.DataFrame'>
Int64Index: 411 entries, 1 to 412
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   congress                       411 non-null    int64  
 1   chamber                        411 non-null    object 
 2   icpsr                          411 non-null    int64  
 3   state_icpsr                    411 non-null    int64  
 4   district_code                  411 non-null    int64  
 5   state_abbrev                   411 non-null    object 
 6   party_code                     411 non-null    int64  
 7   occupancy                      205 non-null    float64
 8   last_means                     205 non-null    float64
 9   bioname                        411 non-null    object 
 10  bioguide_id                    411 non-null    object 
 11  born                           411 non-null    int64  
 12  died                           4 non-null      flo

None

#### nominate_dim1 is the dw nominate score

### Open Secrets

In [96]:
#get key for open secrets api
key_path = "/Users/flatironschool/.secret/open_secrets_api.json"
keys = get_keys(key_path)

api_key = keys['api_key']

In [111]:
crp_ids = pd.DataFrame()
for year in range(2012, 2018, 2):
    temp_ids = pd.read_excel('CRP_IDs.xls', sheet_name='Candidate IDs - {}'.format(year), 
                             skiprows=13, usecols=range(1,6))
    temp_ids_senators = temp_ids[temp_ids['DistIDRunFor'].str.contains('[A-Z][A-Z]S\d', regex=True)]
    crp_ids = pd.concat([crp_ids, temp_ids_senators], ignore_index=True)
    
crp_ids.drop_duplicates(subset='CID', keep='last', inplace=True, ignore_index=True)

crp_ids['last_name'] = crp_ids['CRPName'].apply(lambda x: x.split(', ')[0])
crp_ids['first_middle_name'] = crp_ids['CRPName'].apply(lambda x: x.split(', ')[1])
crp_ids.drop('CRPName', axis=1, inplace=True)

display(crp_ids.head())
display(crp_ids.info())

,CID,Party,DistIDRunFor,FECCandID,last_name,first_middle_name
0,N00035496,R,NJS2,S4NJ00235,Eck,Alieta
1,N00035511,R,NJS2,S4NJ00219,Lonegan,Steve
2,N00035096,3,NJS2,S4NJ00276,Olivera,Pablo
3,N00035280,R,MTS2,S4MT00084,Stapleton,Corey C
4,N00035509,R,IAS2,S4IA00103,Young,David


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 812 entries, 0 to 811
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CID                812 non-null    object
 1   Party              812 non-null    object
 2   DistIDRunFor       812 non-null    object
 3   FECCandID          812 non-null    object
 4   last_name          812 non-null    object
 5   first_middle_name  812 non-null    object
dtypes: object(6)
memory usage: 38.2+ KB


None

In [98]:
crp = CRP(api_key)

In [8]:
contribs = crp.candidates.contrib('N00003389', '2014')
contribs

[{'@attributes': {'org_name': 'Blackstone Group',
   'total': '184700',
   'pacs': '0',
   'indivs': '184700'}},
 {'@attributes': {'org_name': 'Goldman Sachs',
   'total': '129025',
   'pacs': '10000',
   'indivs': '119025'}},
 {'@attributes': {'org_name': 'Humana Inc',
   'total': '104500',
   'pacs': '10000',
   'indivs': '94500'}},
 {'@attributes': {'org_name': 'NorPAC',
   'total': '100151',
   'pacs': '-249',
   'indivs': '100400'}},
 {'@attributes': {'org_name': 'Kindred Healthcare',
   'total': '95450',
   'pacs': '10000',
   'indivs': '85450'}},
 {'@attributes': {'org_name': 'JPMorgan Chase & Co',
   'total': '93075',
   'pacs': '10000',
   'indivs': '83075'}},
 {'@attributes': {'org_name': 'Citigroup Inc',
   'total': '87100',
   'pacs': '10000',
   'indivs': '77100'}},
 {'@attributes': {'org_name': 'Alliance Coal',
   'total': '86600',
   'pacs': '5000',
   'indivs': '81600'}},
 {'@attributes': {'org_name': 'Votesane PAC',
   'total': '82000',
   'pacs': '0',
   'indivs': '82

In [11]:
industries = crp.candidates.industries('N00003389', '2014')
industries

[{'@attributes': {'industry_code': 'F07',
   'industry_name': 'Securities & Investment',
   'indivs': '2157107',
   'pacs': '270600',
   'total': '2427707'}},
 {'@attributes': {'industry_code': 'W06',
   'industry_name': 'Retired',
   'indivs': '1447510',
   'pacs': '0',
   'total': '1447510'}},
 {'@attributes': {'industry_code': 'E01',
   'industry_name': 'Oil & Gas',
   'indivs': '728710',
   'pacs': '380599',
   'total': '1109309'}},
 {'@attributes': {'industry_code': 'K01',
   'industry_name': 'Lawyers/Law Firms',
   'indivs': '761864',
   'pacs': '304650',
   'total': '1066514'}},
 {'@attributes': {'industry_code': 'H01',
   'industry_name': 'Health Professionals',
   'indivs': '734995',
   'pacs': '289500',
   'total': '1024495'}},
 {'@attributes': {'industry_code': 'F09',
   'industry_name': 'Insurance',
   'indivs': '647100',
   'pacs': '366500',
   'total': '1013600'}},
 {'@attributes': {'industry_code': 'F10',
   'industry_name': 'Real Estate',
   'indivs': '876975',
   'pacs

In [12]:
sector = crp.candidates.sector('N00003389', '2014')
sector

[{'@attributes': {'sector_name': 'Agribusiness',
   'sectorid': 'A',
   'indivs': '707620',
   'pacs': '531587',
   'total': '1239207'}},
 {'@attributes': {'sector_name': 'Communic/Electronics',
   'sectorid': 'B',
   'indivs': '683400',
   'pacs': '424243',
   'total': '1107643'}},
 {'@attributes': {'sector_name': 'Construction',
   'sectorid': 'C',
   'indivs': '476426',
   'pacs': '192450',
   'total': '668876'}},
 {'@attributes': {'sector_name': 'Defense',
   'sectorid': 'D',
   'indivs': '68950',
   'pacs': '204900',
   'total': '273850'}},
 {'@attributes': {'sector_name': 'Energy/Nat Resource',
   'sectorid': 'E',
   'indivs': '1282860',
   'pacs': '760082',
   'total': '2042942'}},
 {'@attributes': {'sector_name': 'Finance/Insur/RealEst',
   'sectorid': 'F',
   'indivs': '5123131',
   'pacs': '1198450',
   'total': '6321581'}},
 {'@attributes': {'sector_name': 'Health',
   'sectorid': 'H',
   'indivs': '1691266',
   'pacs': '1057523',
   'total': '2748789'}},
 {'@attributes': {'

In [10]:
help(crp.candidates)

Help on CandidatesClient in module crpapi object:

class CandidatesClient(Client)
 |  CandidatesClient(apikey=None, cache='.cache')
 |  
 |  Retrieves and parses information pertaining to current Congressional
 |  legislators.
 |  
 |  Method resolution order:
 |      CandidatesClient
 |      Client
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  contrib(self, cid, cycle=None)
 |  
 |  contrib_by_ind(self, cid, industry, cycle=None)
 |  
 |  get(self, id_code)
 |      id_code may be either a candidate's specific CID, or a two letter
 |      state code, or a four character district code.
 |  
 |  industries(self, cid, cycle=None)
 |  
 |  pfd(self, cid, year=None)
 |  
 |  sector(self, cid, cycle=None)
 |  
 |  summary(self, cid, cycle=None)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Client:
 |  
 |  __init__(self, apikey=None, cache='.cache')
 |      Initialize self.  See help(type(self)) for accurate signa

### MIT Election Lab

Having trouble finding the right structure for this API call for a download.

In [21]:
#get key for dataverse api
key_path = "/Users/flatironschool/.secret/dataverse_api.json"
keys = get_keys(key_path)

api_key = keys['api_key']

In [31]:
server_url = 'https://dataverse.harvard.edu/dataset'
path = '/api/access/datafile/'
data_id = 'data'
# data_id = ':persistentId/?persistentId=doi:10.7910/DVN/PEJ5QU'

# '$SERVER_URL/api/search?q=$QUERY'

# 'GET http://$SERVER/api/access/datafile/:persistentId/?persistentId=doi:10.5072/FK2/J8SJZB'

url = server_url + path + data_id

r = requests.get(url)
print(r)
print(r.url)
# print(r.content)
print(r.text[:1000])

<Response [404]>
https://dataverse.harvard.edu/dataset/api/access/datafile/data
<?xml version='1.0' encoding='UTF-8' ?>
<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en"><head id="j_idt2"><!-- Global site tag (gtag.js) - Google Analytics -->
<script async="async" src="https://www.googletagmanager.com/gtag/js?id=UA-61753334-1"></script>
<script>
  //<![CDATA[
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date()); gtag('config', 'UA-61753334-1');

  window.addEventListener("load", enableAnalyticsEventCapture, false);

  function enableAnalyticsEventCapture() {
    // Download button
    $(document).on("click", ".btn-download", function() {
      var category = $(this).text();
      var label = getFileId($(this));
      gtag('event', 'Download',{'event_category' : category,
                                'event_label' : label});
    });

    // Request Access button
    $(document).on("click", 

### ProPublica

In [57]:
key_path = "/Users/flatironschool/.secret/pro_publica_api.json"
keys = get_keys(key_path)

api_key = keys['api_key']

In [60]:
def find_bill_info_from_list(bill_array, api_key):
    #instantiate Congress object with api key for Pro Publica API
    congress = Congress(api_key)
    
    #instantiate variables for reporting
    bill_sum_issues = []
    bill_sub_issues = []
    bill_amend_issues = []
    bills_completed = 0
    bill_summaries = 0
    bill_subjects = 0
    bill_amendments = 0
    
    #iterate through bills in list to find summary, subjects, and amendments (if any)
    for bill in bill_array:
        bill_num, bill_cong = bill.split('-')
        bill_cong = int(bill_cong)

        #craft a basic json entry should the call return nothing
        no_response_dict = {bill : 'No response'}

        #iterate through bill calls to get bill info
        
        #bill summaries
        try:
            bill_summary = congress.bills.get(bill_num, congress=bill_cong)
            bill_summaries += 1
        except:
            bill_summary = copy.deepcopy(no_response_dict)
            bill_sum_issues.append(bill)
        
        #bill subjects
        try:
            bill_subject = congress.bills.subjects(bill_num, congress=bill_cong)
            bill_subjects += 1
        except:
            bill_subject = copy.deepcopy(no_response_dict)
            bill_sub_issues.append(bill)

        #bill amendments
        try:
            bill_amend = congress.bills.amendments(bill_num, congress=bill_cong)
            bill_amendments += 1
        except:
            bill_amend = copy.deepcopy(no_response_dict)
            bill_amend_issues.append(bill)

        #write all bill parts to own json files    
        with open('pro_pub_jsons/{}_bill_sum.json'.format(bill), 'w') as f:
            json.dump(bill_summary, f)

        with open('pro_pub_jsons/{}_bill_sub.json'.format(bill), 'w') as f:
            json.dump(bill_subject, f)

        with open('pro_pub_jsons/{}_bill_amend.json'.format(bill), 'w') as f:
            json.dump(bill_amend, f)
            
        bills_completed += 1
        time.sleep(1)
    
    print('Bills parsed: ', bills_completed)
    print()
    print('Bill summaries completed: ', bill_summaries)
    print('Bill subjects completed: ', bill_subjects)
    print('Bill amendments completed: ', bill_amendments)
    print()
    print('Bill summaries with issues: ', bill_sum_issues)
    print('Bill subjects with issues: ', bill_sub_issues)
    print('Bill amendments with issues: ', bill_amend_issues)    
    
    return

In [61]:
# find_bill_info_from_list(bills_to_search, api_key)

Bills parsed:  262

Bill summaries completed:  241
Bill subjects completed:  245
Bill amendments completed:  244

Bill summaries with issues:  ['hr3547-113', 's1926-113', 'hr2642-113', 's1963-113', 's540-113', 's25-113', 's1982-113', 's1752-113', 's1917-113', 's1086-113', 'hr3370-113', 'hr1735-114', 's2943-114', 'treatydoc11412-115', 'hr2810-115', 'hr5515-115', 'treatydoc1134-116', 'treatydoc1121-116', 'treatydoc1141-116', 'treatydoc1118-116', 'treatydoc1161-116']
Bill subjects with issues:  ['hr3547-113', 's1926-113', 'hr2642-113', 's1963-113', 's540-113', 's25-113', 's1982-113', 's1752-113', 's1917-113', 's1086-113', 'hr3370-113', 'treatydoc11412-115', 'treatydoc1134-116', 'treatydoc1121-116', 'treatydoc1141-116', 'treatydoc1118-116', 'treatydoc1161-116']
Bill amendments with issues:  ['hjres106-113', 'hr3547-113', 's1926-113', 'hr2642-113', 's1963-113', 's540-113', 's25-113', 's1982-113', 's1752-113', 's1917-113', 's1086-113', 'hr3370-113', 'treatydoc11412-115', 'treatydoc1134-116',

In [87]:
bad_summaries = ['hr3547-113', 's1926-113', 'hr2642-113', 's1963-113', 's540-113', 's25-113', 's1982-113', 's1752-113', 's1917-113', 's1086-113', 'hr3370-113', 'hr1735-114', 's2943-114', 'treatydoc11412-115', 'hr2810-115', 'hr5515-115', 'treatydoc1134-116', 'treatydoc1121-116', 'treatydoc1141-116', 'treatydoc1118-116', 'treatydoc1161-116']

In [89]:
[x for x in bad_summaries if 'treatydoc' not in x]

['hr3547-113',
 's1926-113',
 'hr2642-113',
 's1963-113',
 's540-113',
 's25-113',
 's1982-113',
 's1752-113',
 's1917-113',
 's1086-113',
 'hr3370-113',
 'hr1735-114',
 's2943-114',
 'hr2810-115',
 'hr5515-115']

In [90]:
congress = Congress(api_key)

In [94]:
congress.bills.get('s1086', congress=113)

{'bill_id': 's1086-113',
 'bill_slug': 's1086',
 'congress': '113',
 'bill': 'S.1086',
 'bill_type': 's',
 'number': 'S.1086',
 'bill_uri': 'https://api.propublica.org/congress/v1/113/bills/s1086.json',
 'title': 'A bill to reauthorize and improve the Child Care and Development Block Grant Act of 1990, and for other purposes.',
 'short_title': 'Child Care and Development Block Grant Act of 2014',
 'sponsor_title': 'Sen.',
 'sponsor': 'Barbara A. Mikulski',
 'sponsor_id': 'M000702',
 'sponsor_uri': 'https://api.propublica.org/congress/v1/members/M000702.json',
 'sponsor_party': 'D',
 'sponsor_state': 'MD',
 'gpo_pdf_uri': None,
 'congressdotgov_url': 'https://www.congress.gov/bill/113th-congress/senate-bill/1086',
 'govtrack_url': 'https://www.govtrack.us/congress/bills/113/s1086',
 'introduced_date': '2013-06-03',
 'active': True,
 'last_vote': '2014-11-17',
 'house_passage': '2014-09-15',
 'senate_passage': '2014-03-13',
 'enacted': '2014-11-18',
 'vetoed': None,
 'cosponsors': 7,
 'c

# Working Zone

In [34]:
member_test = congress.members.get('M000355')
member_test

{'id': 'M000355',
 'member_id': 'M000355',
 'first_name': 'Mitch',
 'middle_name': None,
 'last_name': 'McConnell',
 'suffix': None,
 'date_of_birth': '1942-02-20',
 'gender': 'M',
 'url': 'https://www.mcconnell.senate.gov',
 'times_topics_url': 'http://topics.nytimes.com/top/reference/timestopics/people/m/mitch_mcconnell/index.html',
 'times_tag': 'McConnell, Mitch (Per)',
 'govtrack_id': '300072',
 'cspan_id': '2351',
 'votesmart_id': '53298',
 'icpsr_id': '14921',
 'twitter_account': 'McConnellPress',
 'facebook_account': 'mitchmcconnell',
 'youtube_account': None,
 'crp_id': 'N00003389',
 'google_entity_id': '/m/01z6ls',
 'rss_url': 'https://www.mcconnell.senate.gov/public/?a=RSS.Feed',
 'in_office': True,
 'current_party': 'R',
 'most_recent_vote': '2020-03-26',
 'last_updated': '2020-04-22 09:45:55 -0400',
 'roles': [{'congress': '116',
   'chamber': 'Senate',
   'title': 'Senator, 2nd Class',
   'short_title': 'Sen.',
   'state': 'KY',
   'party': 'R',
   'leadership_role': 'Sen

In [121]:
bill_summary = congress.bills.get('sres15', congress=113)
bill_summary

{'bill_id': 'sres15-113',
 'bill_slug': 'sres15',
 'congress': '113',
 'bill': 'S.RES.15',
 'bill_type': 'sres',
 'number': 'S.RES.15',
 'bill_uri': 'https://api.propublica.org/congress/v1/113/bills/sres15.json',
 'title': 'A resolution to improve procedures for the consideration of legislation and nominations in the Senate.',
 'short_title': 'A resolution to improve procedures for the consideration of legislation and nominations in the Senate.',
 'sponsor_title': 'Sen.',
 'sponsor': 'Harry Reid',
 'sponsor_id': 'R000146',
 'sponsor_uri': 'https://api.propublica.org/congress/v1/members/R000146.json',
 'sponsor_party': 'D',
 'sponsor_state': 'NV',
 'gpo_pdf_uri': None,
 'congressdotgov_url': 'https://www.congress.gov/bill/113th-congress/senate-resolution/15',
 'govtrack_url': 'https://www.govtrack.us/congress/bills/113/sres15',
 'introduced_date': '2013-01-24',
 'active': True,
 'last_vote': '2013-01-24',
 'house_passage': None,
 'senate_passage': None,
 'enacted': None,
 'vetoed': None

In [122]:
bill_subject = congress.bills.subjects('sres15', congress=113)
bill_subject

{'congress': '113',
 'bill_id': 'sres15-113',
 'bill_slug': 'sres15',
 'bill_type': 'sres',
 'number': 'S.RES.15',
 'bill_uri': 'https://api.propublica.org/congress/v1/113/bills/sres15.json',
 'url_number': 'sres15',
 'title': 'A resolution to improve procedures for the consideration of legislation and nominations in the Senate.',
 'sponsor_title': 'Sen.',
 'sponsor_id': 'R000146',
 'sponsor_name': 'Harry Reid',
 'sponsor_state': 'NV',
 'sponsor_party': 'D',
 'sponsor_uri': 'https://api.propublica.org/congress/v1/members/R000146.json',
 'introduced_date': '2013-01-24',
 'number_of_cosponsors': 2,
 'committees': '',
 'latest_major_action_date': '2013-01-24',
 'latest_major_action': 'Resolution agreed to in Senate, under the order of 1/24/2012, having achieved 60 votes in the affirmative, without amendment by Yea-Nay Vote. 78 - 16. Record Vote Number: 1. (text: CR S272)',
 'house_passage_vote': None,
 'senate_passage_vote': None,
 'subjects': [{'name': 'Congress', 'url_name': 'congress'}

In [123]:
bill_amend = congress.bills.amendments('sres15', congress=113)
bill_amend

{'congress': '113',
 'bill_id': 'sres15-113',
 'num_results': 1,
 'offset': 0,
 'amendments': [{'amendment_number': 'S.AMDT.3',
   'slug': 'samdt3',
   'sponsor_title': 'Sen.',
   'sponsor': 'Mike Lee',
   'sponsor_id': 'L000577',
   'sponsor_uri': 'https://api.propublica.org/congress/v1/members/L000577.json',
   'sponsor_party': 'R',
   'sponsor_state': 'UT',
   'introduced_date': '2013-01-24',
   'title': 'To amend the Standing Rules of the Senate to reform the filibuster rules to improve the daily process of the Senate.',
   'congressdotgov_url': 'https://www.congress.gov/amendment/113th-congress/senate-amendment/3/text',
   'latest_major_action_date': '2013-01-24',
   'latest_major_action': 'Amendment SA 3 not agreed to in Senate by Voice Vote.'}]}